# Objective

The objective of this project is to build a model that can classify images into one of the three classes: rock, paper, or scissors. Based on that, have our model correctly classify our gestures in real time, so we can play rock, paper, scissors against our computer.

# Dataset Collection

For this assignment, i used the rock, paper, scissors dataset from roboflow. You can check it out [here](https://app.roboflow.com/roroslab/rock-paper-scissors-sxsw-fzp8u/1).

it's already splitted into:
- train (70% ~ 2196 images)
- validation (19% ~ 604 images)
- test (11% ~ 329 images)

## Downloading the dataset

Make sure to create a roboflow account and get your API key when selecting the 'Download Dataset' button -> selecting YOLOv12 format -> Show download code -> paste the Jupyter code snippet below.

<img src="assets/Screenshot 2025-11-12 at 12.33.55 AM.png" width="500">


Alternatively, you can download the dataset zip filefrom [here](https://universe.roboflow.com/roroslab/rock-paper-scissors-sxsw-fzp8u/dataset/1/download/yolov12).

<img src="assets/Screenshot 2025-11-12 at 12.31.39 AM.png" width="500">

# Preprocessing

Our roboflow dataset in YOLOv12 format is already preprocessed:
- ✅ Images are resized and standardized
- ✅ Labels are in YOLO format (normalized coordinates)
- ✅ Train/val/test splits are done

# YOLO Training

1. We install the yolov8 repository and use the model for training.


In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
! uv pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17731, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 17731 (delta 51), reused 30 (delta 29), pack-reused 17642 (from 3)
Receiving objects: 100% (17731/17731), 17.10 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (12040/12040), done.
/Users/rorosaga/Documents/ie_university/year_4/advanced_ai/repos/yolo_rock_paper_scissors/yolov5
Using Python 3.10.18 environment at: /Users/rorosaga/Documents/ie_university/year_4/advanced_ai/repos/yolo_rock_paper_scissors/.venv


/Users/rorosaga/Documents/ie_university/year_4/advanced_ai/repos/yolo_rock_paper_scissors/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


⠹ mpmath==1.3.0                                                                 Resolved 43 packages in 259ms
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/15.08 KiB           
⠙ Preparing packages... (0/1)-------------- 1.34 KiB/15.08 KiB          
Prepared 1 package in 18ms                                                   
████████████████████ [5/5] thop==0.1.1.post2209072238                           Installed 5 packages in 27ms
 + gitdb==4.0.12
 + gitpython==3.1.45
 + setuptools==80.9.0
 + smmap==5.0.2
 + thop==0.1.1.post2209072238


Installing the dataset from roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY_HERE")
project = rf.workspace("roroslab").project("rock-paper-scissors-sxsw-fzp8u")
version = project.version(1)
dataset = version.download("yolov5")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to rock-paper-scissors-1 in yolov8:: 100%|██████████| 6270/6270 [00:00<00:00, 9048.65it/s] 


In [4]:
import yaml
from pathlib import Path

# Fix data.yaml paths (Roboflow uses relative paths that may not work)
dataset_path = Path("rock-paper-scissors-1")
data_yaml_path = dataset_path / "data.yaml"

# Read the current data.yaml
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update paths to be relative to the data.yaml location
data['train'] = str(dataset_path / "train" / "images")
data['val'] = str(dataset_path / "valid" / "images")
data['test'] = str(dataset_path / "test" / "images")

# Write back
with open(data_yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("Fixed data.yaml paths")
print(f"\nTrain: {data['train']}")
print(f"Val: {data['val']}")
print(f"Test: {data['test']}")
print(f"\nClasses: {data['names']}")
print(f"Number of classes: {data['nc']}")


Fixed data.yaml paths

Train: rock-paper-scissors-1/train/images
Val: rock-paper-scissors-1/valid/images
Test: rock-paper-scissors-1/test/images

Classes: ['Paper', 'Rock', 'Scissors']
Number of classes: 3


In [ ]:
!python train.py \
    --img 640 \
    --batch 16 \
    --epochs 10 \
    --data rock-paper-scissors-1/data.yaml \
    --weights yolov5s.pt \
    --cache --device mps

train: weights=yolov5s.pt, cfg=, data=rock-paper-scissors-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=mps, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-447-ge76591cb Python-3.10.18 torch-2.9.0 MPS

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

# Model Evaluation
